# Splunk App for Data Science and Deep Learning - notebook for CNN based time series forecast

This notebook contains an example workflow for how to forecast a univariate time series. 

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

2024-02-01 12:31:38.976129: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)

numpy version: 1.25.2
pandas version: 2.1.1
TensorFlow version: 2.14.0
Keras version: 2.14.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup internet_traffic.csv 
| timechart span=120min avg("bits_transferred") as bits_transferred 
| eval bits_transferred=round(bits_transferred) 
| fit RobustScaler bits_transferred 
| fit MLTKContainer algo=univariate_cnn_forecast mode=stage epochs=100 batch_size=3 RS_bits_transferred _time by RS_bits_transferred into app:internet_traffic_forecast_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("internet_traffic_forecast_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [5]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [6]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("internet_traffic_forecast_model")
print(df[0:1])
print(df.shape)
print(str(param))
param['options']['params']

        _time  RS_bits_transferred
0  1118152800             0.714191
(616, 2)
{'options': {'params': {'algo': 'forecast_rnn_internet_traffic', 'mode': 'stage', 'epochs': '100', 'batch_size': '3'}, 'args': ['RS_bits_transferred', '_time'], 'feature_variables': ['RS_bits_transferred', '_time'], 'split_by': ['RS_bits_transferred'], 'model_name': 'internet_traffic_forecast_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '10000', 'max_distinct_cat_values_for_classifiers': '10000', 'max_distinct_cat_values_for_scoring': '10000', 'max_fit_time': '6000', 'max_inputs': '10000000', 'max_memory_usage_mb': '16000', 'max_model_size_mb': '3000', 'max_score_time': '6000', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['RS_bits_transferred', '_time']}


{'algo': 'forecast_rnn_internet_traffic',
 'mode': 'stage',
 'epochs': '100',
 'batch_size': '3'}

## Stage 2 - create and initialize a model

In [7]:
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    # Collect variables
    model_batch_size = 3
    n_features = 1
    hidden_layers = 50
    activation_func = 'sigmoid'
    if 'options' in param:
        if 'params' in param['options']:
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
            if 'hidden_layers' in param['options']['params']:
                hidden_layers = int(param['options']['params']['hidden_layers'])
            if 'activation' in param['options']['params']:
                activation_func = param['options']['params']['activation']
    
    # define model
    model = keras.Sequential()
    model.add(keras.layers.Conv1D(filters=64, kernel_size=2, activation=activation_func, input_shape=(model_batch_size, n_features)))
    model.add(keras.layers.MaxPooling1D(pool_size=2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(hidden_layers,activation=activation_func))
    model.add(keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [8]:
# test mltkc_stage_create_model
model = init(df,param)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2, 64)             192       
                                                                 
 max_pooling1d (MaxPooling1  (None, 1, 64)             0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 50)                3250      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3493 (13.64 KB)
Trainable params: 3493 (13.64 KB)
Non-trainable params: 0 (0.00 Byte)
______________________

## Stage 3 - fit the model

In [9]:
# returns a fit info json object
# split a univariate sequence into samples
def split_sequence(sequence, batch_size):
    
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + batch_size
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def fit(model,df,param):
    returns = {}
    
    # Collect variables from param file
    model_epochs = 100
    model_batch_size = 3
    holdback = 30
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
            if 'holdback' in param['options']['params']:
                holdback = int(param['options']['params']['holdback'])
    
    
    # flatten data frame into an array and extract the training set
    full_data = df[param['options']['split_by']].values.tolist()
    train_set = list(full_data[:len(full_data)-holdback])
    
    # split data into samples
    X, y = split_sequence(train_set, model_batch_size)
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], 1))
    

    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    # run the training
    returns['fit_history'] = model.fit(x=X,
                                       y=y, 
                                       verbose=2, 
                                       epochs=model_epochs,
                                       shuffle=False,
                                       callbacks=[tensorboard_callback])
    # memorize parameters
    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    returns['model_loss_acc'] = model.evaluate(x = X, y = y)
    return returns

In [10]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

Epoch 1/100
19/19 - 1s - loss: 0.9331 - 689ms/epoch - 36ms/step
Epoch 2/100
19/19 - 0s - loss: 0.4053 - 90ms/epoch - 5ms/step
Epoch 3/100
19/19 - 0s - loss: 0.3465 - 134ms/epoch - 7ms/step
Epoch 4/100
19/19 - 0s - loss: 0.3392 - 83ms/epoch - 4ms/step
Epoch 5/100
19/19 - 0s - loss: 0.3308 - 98ms/epoch - 5ms/step
Epoch 6/100
19/19 - 0s - loss: 0.3228 - 81ms/epoch - 4ms/step
Epoch 7/100
19/19 - 0s - loss: 0.3149 - 64ms/epoch - 3ms/step
Epoch 8/100
19/19 - 0s - loss: 0.3067 - 81ms/epoch - 4ms/step
Epoch 9/100
19/19 - 0s - loss: 0.2983 - 78ms/epoch - 4ms/step
Epoch 10/100
19/19 - 0s - loss: 0.2899 - 92ms/epoch - 5ms/step
Epoch 11/100
19/19 - 0s - loss: 0.2815 - 88ms/epoch - 5ms/step
Epoch 12/100
19/19 - 0s - loss: 0.2732 - 79ms/epoch - 4ms/step
Epoch 13/100
19/19 - 0s - loss: 0.2652 - 83ms/epoch - 4ms/step
Epoch 14/100
19/19 - 0s - loss: 0.2576 - 87ms/epoch - 5ms/step
Epoch 15/100
19/19 - 0s - loss: 0.2505 - 83ms/epoch - 4ms/step
Epoch 16/100
19/19 - 0s - loss: 0.2437 - 77ms/epoch - 4ms/ste

## Stage 4 - apply the model

In [11]:
def apply(model,df,param):
    
    # Collect variables
    model_batch_size = 3
    future_steps = 30
    holdback = 30
    if 'options' in param:
        if 'params' in param['options']:
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
            if 'future_steps' in param['options']['params']:
                future_steps = int(param['options']['params']['future_steps'])
            if 'holdback' in param['options']['params']:
                holdback = int(param['options']['params']['holdback'])
    
    # select training data
    X = df[param['options']['split_by']].values

    test_set = X[len(X)-holdback-model_batch_size:]
    predictions = list(X[:len(X)-holdback])
    # generate forecast
    for i in range(0, holdback+future_steps):
        if i<holdback:
            X_batch = test_set[i:i+model_batch_size].reshape(1,model_batch_size,1)
            y_pred = model.predict(x = X_batch, verbose=1)
            predictions.append(list(y_pred[0]))
        else:
            X_batch = test_set[i:i+model_batch_size].reshape(1,model_batch_size,1)
            y_pred = model.predict(x = X_batch, verbose=1)
            predictions.append(list(y_pred[0]))
            test_set = np.append(test_set, y_pred[0])
            
    # append predictions to time series to return a data frame
    return predictions

In [12]:
training_instance = apply(model,df,param)
print(len(training_instance))
training_instance[583:]

1/1 [==============================] - 0s 17ms/step
646


[array([0.328399]),
 array([0.18502462]),
 array([-9.24839172e-05]),
 [-0.0953756],
 [-0.2724685],
 [-0.36768532],
 [-0.25154704],
 [0.13848017],
 [1.0742664],
 [1.2467495],
 [0.9159776],
 [0.5980304],
 [0.4464996],
 [-0.2050779],
 [-0.19790936],
 [-0.10597366],
 [-0.29441327],
 [-0.36397767],
 [-0.22809334],
 [0.35760197],
 [1.1380372],
 [1.1604118],
 [0.7387321],
 [0.628437],
 [0.27152884],
 [-0.26238215],
 [-0.20401311],
 [-0.17258857],
 [-0.33285034],
 [-0.30834076],
 [-0.17403547],
 [0.46485665],
 [1.1610049],
 [1.1802484],
 [0.8335834],
 [0.6336271],
 [0.13165574],
 [-0.021384852],
 [-0.2857948],
 [-0.2622921],
 [-0.21963409],
 [-0.07609786],
 [0.09566988],
 [0.24868257],
 [0.33310157],
 [0.33902824],
 [0.28239062],
 [0.19052932],
 [0.08408539],
 [-0.008798974],
 [-0.07755573],
 [-0.10699496],
 [-0.09610684],
 [-0.04688935],
 [0.027080877],
 [0.101846986],
 [0.15443543],
 [0.17117937],
 [0.15777183],
 [0.12082516],
 [0.075496376],
 [0.028129516],
 [-0.007014947]]

## Stage 5 - save the model

In [13]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model to keras file
    model.save(MODEL_DIRECTORY + name + ".keras")
    return model

## Stage 6 - load the model

In [14]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = keras.models.load_model(MODEL_DIRECTORY + name + ".keras")
    return model

## Stage 7 - provide a summary of the model

In [15]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        model.summary(print_fn=lambda x: s.append(x+'\n'))
        returns["summary"] = ''.join(s)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code